<a href="https://colab.research.google.com/github/AnastasiiaVladimirovna/transcriptomics_homeworks/blob/main/hw14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scanpy[leiden] anndata2ri scikit-misc scvi-tools harmonypy

# Необходимо, чтобы конкретно прогрузился matplotlib
import os
os.kill(os.getpid(), 9)

In [ ]:
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

# Настраиваем эститику иллюстраций
sc.settings.set_figure_params(dpi=150)
sc.settings.verbosity = 3
sns.set_style("ticks")

adata = sc.datasets.pbmc3k()

try downloading from url
http://falexwolf.de/data/pbmc3k_raw.h5ad
... this may take a while but only happens once


  0%|          | 0.00/5.58M [00:00<?, ?B/s]

## Часть 2

In [ ]:
adata.write_h5ad("Azimuth.h5ad")

In [ ]:
!gdown "1jW548g6ERFS0t7NywgyjRs6VaE5QwXbg&confirm=t"
adata_reference = sc.read_h5ad("PBMC_Satija.h5ad")
adata_reference.layers["counts"] = adata_reference.X.copy()
adata_reference.obs["ref_query"] = "reference"

Downloading...
From: https://drive.google.com/uc?id=1jW548g6ERFS0t7NywgyjRs6VaE5QwXbg&confirm=t
To: /content/PBMC_Satija.h5ad
100% 243M/243M [00:01<00:00, 130MB/s]


In [ ]:
import scanpy.external as sce

In [ ]:
def prediction(adata, adata_reference):
  #готовим обе таблицы и объединяем их
  adata.obs["ref_query"] = "query"
  adata.obs["donor"] = "query"
  # adata.obs.index = [f"{bc.split('-')[0]}:query" for bc in adata.obs.index]
  # adata_reference.obs.index = [f"{bc.split('_')[1]}:{bc.split('_')[0]}" for bc in adata_reference.obs.index]
  adata_concat = adata.concatenate(adata_reference, index_unique=None)
  adata_concat.layers["counts"] = adata_concat.X.copy()
  # del adata_concat.obs["batch"]

  #интеграция при помощи Harmony
  sc.pp.normalize_total(adata_concat)
  sc.pp.log1p(adata_concat)
  sc.pp.highly_variable_genes(
    adata_concat,
    n_top_genes=3000,
    flavor="seurat_v3",
    layer="counts",
    batch_key="donor"
    )
  sc.pp.scale(adata_concat)
  sc.tl.pca(adata_concat)
  sc.external.pp.harmony_integrate(adata_concat, key="donor", max_iter_harmony=20)
  sc.pp.neighbors(
    adata_concat,
    use_rep="X_pca_harmony",
    n_pcs=30
    )
  sc.tl.umap(adata_concat)

  #Строим kNN-граф
  k = 20 
  sc.pp.neighbors(adata_concat, use_rep="X_pca_harmony", n_pcs=30, n_neighbors=k, key_added="prediction")

  #Делаем колонку с типами клеток
  celltype_predicted = []
  # prediction_score = []
  counter = 0
  for bc in adata_concat.obs_names:
    if adata_concat.obs.loc[bc]["ref_query"] == "reference":
      celltype_predicted.append(float("NaN"))
      # prediction_score.append(float("NaN"))
    else:
      neighbors_celltype = adata_concat.obs["celltype.l1"][(adata_concat.obsp["prediction_distances"][counter] > 0).A[0]]
      if (~neighbors_celltype.isnull()).sum() == 0:
        celltype_predicted.append("Unknown")
        # prediction_score.append(float("NaN"))
      else:
        neighbors_celltype = neighbors_celltype[~neighbors_celltype.isnull()]
        ct = neighbors_celltype.value_counts().index[0]
        score = neighbors_celltype.value_counts()[0] / len(neighbors_celltype)
        # prediction_score.append(score)
        celltype_predicted.append(ct)
    counter += 1
  adata_concat.obs["celltype_predicted"] = celltype_predicted
  # adata_concat.obs["prediction_score"] = prediction_score
  return adata_concat[list(adata_concat.obs['ref_query'] == 'query')]

In [ ]:
adata_1 = prediction(adata, adata_reference)

/usr/local/lib/python3.8/dist-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


normalizing counts per cell
    finished (0:00:00)
If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
--> added
    'highly_variable', boolean vector (adata.var)
    'highly_variable_rank', float vector (adata.var)
    'means', float vector (adata.var)
    'variances', float vector (adata.var)
    'variances_norm', float vector (adata.var)
... as `zero_center=True`, sparse input is densified and may lead to large memory consumption
computing PCA
    on highly variable genes
    with n_comps=50
    finished (0:00:14)


2022-12-25 21:51:52,298 - harmonypy - INFO - Computing initial centroids with sklearn.KMeans...
INFO:harmonypy:Computing initial centroids with sklearn.KMeans...
2022-12-25 21:52:12,667 - harmonypy - INFO - sklearn.KMeans initialization complete.
INFO:harmonypy:sklearn.KMeans initialization complete.
2022-12-25 21:52:12,898 - harmonypy - INFO - Iteration 1 of 20
INFO:harmonypy:Iteration 1 of 20
2022-12-25 21:52:27,446 - harmonypy - INFO - Iteration 2 of 20
INFO:harmonypy:Iteration 2 of 20
2022-12-25 21:52:39,267 - harmonypy - INFO - Iteration 3 of 20
INFO:harmonypy:Iteration 3 of 20
2022-12-25 21:52:51,145 - harmonypy - INFO - Iteration 4 of 20
INFO:harmonypy:Iteration 4 of 20
2022-12-25 21:53:02,920 - harmonypy - INFO - Iteration 5 of 20
INFO:harmonypy:Iteration 5 of 20
2022-12-25 21:53:14,700 - harmonypy - INFO - Iteration 6 of 20
INFO:harmonypy:Iteration 6 of 20
2022-12-25 21:53:26,675 - harmonypy - INFO - Converged after 6 iterations
INFO:harmonypy:Converged after 6 iterations


computing neighbors
    finished: added to `.uns['neighbors']`
    `.obsp['distances']`, distances for each pair of neighbors
    `.obsp['connectivities']`, weighted adjacency matrix (0:00:03)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:26)
computing neighbors
    finished: added to `.uns['prediction']`
    `.obsp['prediction_distances']`, distances for each pair of neighbors
    `.obsp['prediction_connectivities']`, weighted adjacency matrix (0:00:04)


In [ ]:
adata_1.obs["celltype_predicted"]

AAACATACAACCAC-1    CD8 T
AAACATTGAGCTAC-1        B
AAACATTGATCAGC-1    CD4 T
AAACCGTGCTTCCG-1     Mono
AAACCGTGTATGCG-1       NK
                    ...  
TTTCGAACTCTCAT-1     Mono
TTTCTACTGAGGCA-1        B
TTTCTACTTCCTCG-1        B
TTTGCATGAGAGGC-1        B
TTTGCATGCCTCAC-1    CD4 T
Name: celltype_predicted, Length: 2700, dtype: object